In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

/home/felix/.local/share/virtualenvs/Comparing-Measures-of-LLM-Truthfulness-c5kw3004/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from simpleTQA import SimpleTQA
from RepresentationEngineering.repe import repe_pipeline_registry
repe_pipeline_registry()

from RepresentationEngineering.examples.honesty.utils import honesty_function_dataset, plot_lat_scans, plot_detection_results 

### Setup

In [ ]:
model_path = "meta-llama/Llama-2-7b-chat-hf"
model = "llama-2-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
rep_token = -1
hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1))
n_difference = 1
direction_method = "pca"
rep_reading_pipeline = pipeline("rep-reading", model=model, tokenizer=tokenizer)

In [ ]:
user_tag = "USER:"
assistant_tag = "ASSISTANT:"

data_path = "RepresentationEngineering/data/facts/facts_true_false.csv"
dataset = honesty_function_dataset(data_path, tokenizer, user_tag, assistant_tag)

### Get honesty direction using representation reading

In [ ]:
honesty_rep_reader = rep_reading_pipeline.get_directions(
    dataset['train']['data'], 
    rep_token=rep_token, 
    hidden_layers=hidden_layers, 
    n_difference=n_difference, 
    train_labels=dataset['train']['labels'], 
    direction_method=direction_method,
    batch_size=32,
)

In [ ]:
H_tests = rep_reading_pipeline(
    dataset['test']['data'], 
    rep_token=rep_token, 
    hidden_layers=hidden_layers, 
    rep_reader=honesty_rep_reader,
    batch_size=32)

In [ ]:
def get_rep_readings(completion, important_layers=None):
    if important_layers is None:
        important_layers = hidden_layers
    input_ids = tokenizer.tokenize(completion)
    results = []

    for rep_token_pos in range(-len(input_ids), 0, 1):
        H_tests = rep_reading_pipeline(
            completion,
            rep_reader=honesty_rep_reader,
            rep_token=rep_token_pos,
            hidden_layers=hidden_layers
        )
        results.append(H_tests)

    honesty_scores = []
    honesty_scores_means = []
    for pos in range(len(results)):
        tmp_scores = []
        tmp_scores_all = []
        for layer in hidden_layers:
            tmp_scores_all.append(
                results[pos][0][layer][0] * honesty_rep_reader.direction_signs[layer][0]
            )
            if layer in important_layers:
                tmp_scores.append(
                    results[pos][0][layer][0] * honesty_rep_reader.direction_signs[layer][0]
                )
        honesty_scores.append(tmp_scores_all)
        honesty_scores_means.append(np.mean(tmp_scores))

    return honesty_scores, honesty_scores_means


In [ ]:
simple_tqa = SimpleTQA()

In [1]:
honesty_scores_truth = []
honesty_score_means_t = []
honesty_scores_lie = []
honesty_score_means_l = []

In [ ]:
for answer in simple_tqa[f"{model}_answers_truth"]:
    honesty_scores, honesty_scores_means = get_rep_readings(answer, important_layers=None)
    honesty_scores_truth.append(honesty_scores)
    honesty_score_means_t.append(honesty_scores_means)

In [ ]:
for answer in simple_tqa[f"{model}_answers_lie"]:
    honesty_scores, honesty_scores_means = get_rep_readings(answer, important_layers=None)
    honesty_scores_lie.append(honesty_scores)
    honesty_score_means_l.append(honesty_scores_means)

### Get lie detector probabilities on simpleTQA

In [ ]:
# get rows for which both lie and truth logprobs were generated
logprobs_dataset = simple_tqa[simple_tqa[f"{model}_logprobs_difference_lie"].notna()]
logprobs_dataset = logprobs_dataset[logprobs_dataset[f"{model}_logprobs_difference_truth"].notna()]

In [ ]:
logprobs_dataset[f"{model}_logprobs_difference_lie"] = logprobs_dataset[f"{model}_logprobs_difference_lie"].apply(lambda x: np.array(x))
logprobs_dataset[f"{model}_logprobs_difference_truth"] = logprobs_dataset[f"{model}_logprobs_difference_truth"].apply(lambda x: np.array(x))

In [ ]:
logprob_diffs = np.concatenate([logprobs_dataset[f"{model}_logprobs_difference_lie"], logprobs_dataset[f"{model}_logprobs_difference_truth"]])
labels = np.concatenate([np.zeros(len(logprobs_dataset[f"{model}_logprobs_difference_lie"])), np.ones(logprobs_dataset[f"{model}_logprobs_difference_truth"])])

In [ ]:
with open(f"trained_classifiers/logistic_logprobs_classifier.pkl", "rb") as f:
    classifier = pickle.load(f)

In [ ]:
accuracy, auc, _, lie_detector_pred, lie_detector_pred_proba = classifier.evaluate(logprob_diffs, labels, return_ys=True)

In [ ]:
lie_detector_pred_proba = np.array(lie_detector_pred_proba)

### Compare results from RepEng with lie detector

In [ ]:
# Sample data
x1 = lie_detector_pred_proba[labels == 1]
y1 = honesty_score_means_t

x2 = lie_detector_pred_proba[labels == 0]
y2 = honesty_score_means_l

# Create a scatter plot with different colors and labels
plt.scatter(x1, y1, c='blue', label='Truth')
plt.scatter(x2, y2, c='red', label='Lie')

# Add labels and title
plt.xlabel('Log(P())')
plt.ylabel('Y-axis Label')
plt.title('Scatter Plot with Different Colors')

# Add a legend
plt.legend()

# Display the plot
plt.show()